<a href="https://colab.research.google.com/github/JPA-BERT/ShinAsakawa.github.io/blob/master/notebooks/2020_0722sequence_models_tutorial_ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch tutorial の内容について(4)

- date: 2020-0722
- author: 浅川伸一

[https://github.com/pytorch/tutorials/tree/master/beginner_source/nlp](https://github.com/pytorch/tutorials/tree/master/beginner_source/nlp) を見ると
PyTorch で 自然言語処理を行う場合のチュートリアルは以下とおりである

# Deep Learning for NLP with Pytorch

1. [pytorch_tutorial.py](https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/pytorch_tutorial.py): 
	[PyTorch 入門 Introduction to PyTorch](https://pytorch.org/tutorials/beginner/nlp/pytorch_tutorial.html)

2. [deep_learning_tutorial.py](https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/deep_learning_tutorial.py): 
	[PyTorch による深層学習 Deep Learning with PyTorch](https://pytorch.org/tutorials/beginner/nlp/deep_learning_tutorial.html)

3. [word_embeddings_tutorial.py](https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/word_embeddings_tutorial.py): 
	[単語埋め込み:語彙的意味の符号化 Word Embeddings: Encoding Lexical Semantics](https://pytorch.org/tutorials/beginner/nlp/word_embeddings_tutorial.html)

4. [sequence_models_tutorial.py]((https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/sequence_models_tutorial.py): 
	[系列モデルと LSTM Sequence Models and Long-Short Term Memory Networks](https://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html)

5. [advanced_tutorial.py]((https://github.com/pytorch/tutorials/blob/master/beginner_source/nlp/advanced_tutorial.py): 
	[動的意思決定と双方向 LSTM 条件付き確率場 Advanced: Making Dynamic Decisions and the Bi-LSTM CRF](https://pytorch.org/tutorials/beginner/nlp/advanced_tutorial.html)


以下では，このうちの 4 について解説している。


In [ ]:
%matplotlib inline

# 系列モデルと LSTM ネットワーク
ここまで，様々なフィードフォワードネットワークを見てきました。
フィードフォワードニューラルネットワークは，ネットワークが保持する(内部) 状態がありません。
フィードフォワードモデルは，系列処理に向いているモデルではないと言えます。
一方，系列モデルは 自然言語処理 (NLP) の中核をなすモデルです。
系列モデルとは，入力情報間に時間的依存関係を仮定したモデルです。
系列モデルの古典的な例としては，品詞タグ付けのための ``隠れマルコフモデル`` があります。
他の例としては ``条件付き確率場`` があります。


<!--
# Sequence Models and Long-Short Term Memory Networks

At this point, we have seen various feed-forward networks. 
That is, there is no state maintained by the network at all. 
This might not be the behavior we want. 
Sequence models are central to NLP: they are models where there is some sort of dependence through time between your inputs. 
The classical example of a sequence model is the Hidden Markov Model for part-of-speech tagging. 
Another example is the conditional random field.
-->

リカレント・ニューラルネットワークは，状態を保持することができるネットワークです。
例えば，ネットワークの出力は次の入力の一部として使用されます。
リカレントニューラルネットワークは，全系列を通して情報が伝播するようにすることもできます。
LSTM の場合 系列の各要素に対して，対応する *隠れ状態* $h_t$ があり，原則として，系列の任意の時点の情報を蓄えておくことができます。
隠れ状態を利用して，言語モデルの単語や品詞タグ付，その他の多くの予測することが可能です。

<!--
A recurrent neural network is a network that maintains some kind of state. 
For example, its output could be used as part of the next input, so that information can propogate along as the network passes over the sequence. In the case of an LSTM, for each element in the sequence, there is a corresponding *hidden state* $h_t$, which in principle can contain information from arbitrary points earlier in the sequence. 
We can use the hidden state to predict words in a language model, part-of-speech tags, and a myriad of other things.
-->

### PyTorch における LSTM

例題に入る前に いくつかのことに注意してください。
Pytorch の LSTM は、すべての入力が 3 次元テンソルであることを仮定しています。
この入力テンソルの軸の意味は重要です。
第 1 軸(次元)は系列そのものです。
第 2 軸(次元) はミニバッチ内の実体，
第 3 軸(次元) の軸は入力の要素を表します。
ミニバッチについてはここでは議論していません。
ですので ミニバッチは無視して，第 2 軸には常に 1つの次元があると仮定しましょう。
``牛が飛び跳ねた The cow jumped`` という文に対して系列モデルを実行したい場合，
入力は以下のようになります:

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{行ベクトル} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}
\end{align}

サイズ 1 の追加した第 2 次元があることを忘れないでください。

さらに 系列を一度に 1 つずつ進めることもできます。ですが その場合 第1軸 もサイズが 1 になります。

簡単な例を見てみましょう。

<!--
### LSTM's in Pytorch

Before getting to the example, note a few things. Pytorch's LSTM expects all of its inputs to be 3D tensors. The semantics of the axes of these tensors is important. The first axis is the sequence itself, the second indexes instances in the mini-batch, and the third indexes elements of the input. 
We haven't discussed mini-batching, so lets just ignore that and assume we will always have just 1 dimension on the second axis. 
If we want to run the sequence model over the sentence "The cow jumped", our input should look like

\begin{align}\begin{bmatrix}
   \overbrace{q_\text{The}}^\text{row vector} \\
   q_\text{cow} \\
   q_\text{jumped}
   \end{bmatrix}\end{align}

Except remember there is an additional 2nd dimension with size 1.

In addition, you could go through the sequence one at a time, in which case the 1st axis will have size 1 also.

Let's see a quick example.
-->


In [ ]:
# Author: Robert Guthrie

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

torch.manual_seed(1)

In [ ]:
lstm = nn.LSTM(3, 3)  # 入力次元は 3 出力次元も 3 です
inputs = [torch.randn(1, 3) for _ in range(5)]  # 長さ 5 の系列を作成

# 隠れ状態の初期化
hidden = (torch.randn(1, 1, 3),
          torch.randn(1, 1, 3))
for i in inputs:
    # 全系列を通じて，一度に一つづづ，逐次，Step through the sequence one element at a time.
    # 1ステップ後，中間層は，中間層の状態と入力情報とが入力となる # after each step, hidden contains the hidden state.
    out, hidden = lstm(i.view(1, 1, -1), hidden)


# 代わりに 系列全体を一度に実行することもできます。
# LSTM が返す最初の値は  系列全体の 隠れ状態のすべてです。
# 2 番目の値は 最新の隠れ状態です
#（"out "と "hidden "の最後のスライスを比較してみてください。
# その理由は次のようになります:
# "out "は 系列内の全隠れ状態にアクセスできるようにします。
#  "hidden" は 系列を継続して誤差逆伝播することを可能にします。
# 後で LSTM に引数として渡すことで 2 次元を追加します。
# alternatively, we can do the entire sequence all at once.
# the first value returned by LSTM is all of the hidden states throughout
# the sequence. the second is just the most recent hidden state
# (compare the last slice of "out" with "hidden" below, they are the same)
# The reason for this is that:
# "out" will give you access to all hidden states in the sequence
# "hidden" will allow you to continue the sequence and backpropagate,
# by passing it as an argument  to the lstm at a later time
# Add the extra 2nd dimension
inputs = torch.cat(inputs).view(len(inputs), 1, -1)
hidden = (torch.randn(1, 1, 3), torch.randn(1, 1, 3))  # clean out hidden state
out, hidden = lstm(inputs, hidden)
print(out)
print(hidden)

tensor([[[-0.0187,  0.1713, -0.2944]],

        [[-0.3521,  0.1026, -0.2971]],

        [[-0.3191,  0.0781, -0.1957]],

        [[-0.1634,  0.0941, -0.1637]],

        [[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>)
(tensor([[[-0.3368,  0.0959, -0.0538]]], grad_fn=<StackBackward>), tensor([[[-0.9825,  0.4715, -0.0633]]], grad_fn=<StackBackward>))


### 例: 品詞タグ付け LSTM
<!--
### Example: An LSTM for Part-of-Speech Tagging
-->

<!--
In this section, we will use an LSTM to get part of speech tags. 
We will not use Viterbi or Forward-Backward or anything like that, but as a (challenging) exercise to the reader, think about how Viterbi could be used after you have seen what is going on.
-->

ここでは LSTM を使って品詞タグ付けを行います。
ビタビ (アルゴリズム) や 前向き-後ろ向き (アルゴリズム) などは使いません。
ですが，読者への (挑戦的な) 練習問題として，説明を読んだ後で，どうすれば ビタビ アルゴリズムが使えるかを考えてみてください。

<!--
The model is as follows: let our input sentence be $w_1, \dots, w_M$, where $w_i \in V$, our vocab. Also, let $T$ be our tag set, and $y_i$ the tag of word $w_i$.
Denote our prediction of the tag of word $w_i$ by $\hat{y}_i$.
-->

モデルは以下のとおりです:
入力系列を語彙の系列 $w_1, \ldots, w_M$ とします。
また $T$ をタグ集合，$y_i$ を単語 $w_i$ に対応するタグとします。
単語 $w_i$ の品詞の予測値は $\hat{y}_i$ とします。

これは，予測モデルであり出力は $\hat{y}_1,\ldots,\hat{y}_M$ とします。ここで $\hat{y}_i \in T$ とします。

<!--This is a structure prediction, model, where our output is a sequence $\hat{y}_1, \dots, \hat{y}_M$, where $\hat{y}_i \in T$.-->

予測を行う際には 文の上を LSTM を入力します。
時刻 $i$ での中間層状態を$h_i$ とします。
また それぞれのタグに一意なインデックスを割り当てる(単語の埋め込みの項で word_to\_ixとしたように)。
すると $hat{y}_i$ の予測は以下のようになります:

<!--
To do the prediction, pass an LSTM over the sentence. Denote the hidden state at timestep $i$ as $h_i$. Also, assign each tag a unique index (like how we had word\_to\_ix in the word embeddings section). Then our prediction rule for $\hat{y}_i$ is
-->

\begin{align}\hat{y}_i = \text{argmax}_j \  (\log \text{Softmax}(Ah_i + b))_j\end{align}

<!--
That is, take the log softmax of the affine map of the hidden state, and the predicted tag is the tag that has the maximum value in this vector. 
Note this implies immediately that the dimensionality of the target space of $A$ is $|T|$.
-->

すなわち，中間層状態のアフィン写像の 対数ソフトマックスから，このベクトル中の最大値を持つタグを予測タグとします。
このことは $A$ のターゲット空間の次元が $|T|$ であることを暗示していることに注意してください。

データの準備:
<!--Prepare data:-->



In [ ]:
def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


training_data = [
    ("The dog ate the apple".split(), ["DET", "NN", "V", "DET", "NN"]),
    ("Everybody read that book".split(), ["NN", "V", "DET", "NN"])
]
word_to_ix = {}
for sent, tags in training_data:
    for word in sent:
        if word not in word_to_ix:
            word_to_ix[word] = len(word_to_ix)
print(word_to_ix)
tag_to_ix = {"DET": 0, "NN": 1, "V": 2}

# These will usually be more like 32 or 64 dimensional.
# We will keep them small, so we can see how the weights change as we train.
EMBEDDING_DIM = 6
HIDDEN_DIM = 6

<!--Create the model:-->
モデルの作成:



In [ ]:
class LSTMTagger(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, tagset_size):
        super(LSTMTagger, self).__init__()
        self.hidden_dim = hidden_dim

        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # The LSTM takes word embeddings as inputs, and outputs hidden states
        # with dimensionality hidden_dim.
        self.lstm = nn.LSTM(embedding_dim, hidden_dim)

        # The linear layer that maps from hidden state space to tag space
        self.hidden2tag = nn.Linear(hidden_dim, tagset_size)

    def forward(self, sentence):
        embeds = self.word_embeddings(sentence)
        lstm_out, _ = self.lstm(embeds.view(len(sentence), 1, -1))
        tag_space = self.hidden2tag(lstm_out.view(len(sentence), -1))
        tag_scores = F.log_softmax(tag_space, dim=1)
        return tag_scores

Train the model:



In [ ]:
model = LSTMTagger(EMBEDDING_DIM, HIDDEN_DIM, len(word_to_ix), len(tag_to_ix))
loss_function = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1)

# See what the scores are before training
# Note that element i,j of the output is the score for tag j for word i.
# Here we don't need to train, so the code is wrapped in torch.no_grad()
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)
    print(tag_scores)

for epoch in range(300):  # again, normally you would NOT do 300 epochs, it is toy data
    for sentence, tags in training_data:
        # Step 1. Remember that Pytorch accumulates gradients.
        # We need to clear them out before each instance
        model.zero_grad()

        # Step 2. Get our inputs ready for the network, that is, turn them into
        # Tensors of word indices.
        sentence_in = prepare_sequence(sentence, word_to_ix)
        targets = prepare_sequence(tags, tag_to_ix)

        # Step 3. Run our forward pass.
        tag_scores = model(sentence_in)

        # Step 4. Compute the loss, gradients, and update the parameters by
        #  calling optimizer.step()
        loss = loss_function(tag_scores, targets)
        loss.backward()
        optimizer.step()

# See what the scores are after training
with torch.no_grad():
    inputs = prepare_sequence(training_data[0][0], word_to_ix)
    tag_scores = model(inputs)

    # The sentence is "the dog ate the apple".  i,j corresponds to score for tag j
    # for word i. The predicted tag is the maximum scoring tag.
    # Here, we can see the predicted sequence below is 0 1 2 0 1
    # since 0 is index of the maximum value of row 1,
    # 1 is the index of maximum value of row 2, etc.
    # Which is DET NOUN VERB DET NOUN, the correct sequence!
    print(tag_scores)

### Exercise: Augmenting the LSTM part-of-speech tagger with character-level features

In the example above, each word had an embedding, which served as the inputs to our sequence model. Let's augment the word embeddings with a representation derived from the characters of the word. 
We expect that this should help significantly, since character-level information like affixes have a large bearing on part-of-speech. 
For example, words with the affix *-ly* are almost always tagged as adverbs in English.

To do this, let $c_w$ be the character-level representation of word $w$. 
Let $x_w$ be the word embedding as before. 
Then the input to our sequence model is the concatenation of $x_w$ and $c_w$. 
So if $x_w$ has dimension 5, and $c_w$ dimension 3, then our LSTM should accept an input of dimension 8.

To get the character level representation, do an LSTM over the characters of a word, and let $c_w$ be the final hidden state of this LSTM. Hints:

* There are going to be two LSTM's in your new model.
  The original one that outputs POS tag scores, and the new one that
  outputs a character-level representation of each word.
* To do a sequence model over characters, you will have to embed characters.
  The character embeddings will be the input to the character LSTM.

